In [1]:
from bs4 import BeautifulSoup
import requests
import re
import googlemaps
import csv
import json
import numpy as np

In [2]:
#import en_core_web_sm
#from flashgeotext.geotext import GeoText
#spacy.load('en_core_web_lg')

In [1]:
from geotext import GeoText

In [4]:
#from geopy.geocoders import Nominatim
#from geopy.exc import GeocoderTimedOut

In [5]:
#import subprocess
#import sys
#def install(package):
#    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [6]:
#install('flashgeotext')

In [7]:
#subprocess.check_call([sys.executable, "-m", "spacy","download","en_core_web_lg"])

In [8]:
gmaps = googlemaps.Client(key=gmapKey)

In [9]:
mfaId = '17'
itemId = 'gty_'+mfaId
tags = []

In [10]:
website = requests.get('http://www.getty.edu/art/collection/objects/'+mfaId)
print(website)

<Response [200]>


In [11]:
soup = BeautifulSoup(website.content, 'html.parser')

In [20]:
#print(soup.prettify())

In [13]:
yearStr = []
yearNum = []
for date in soup.find_all("section", {"id":"object-section-provenance"})[0].find_all("div", {"class":"small-12 medium-3 large-3 columns"}):
    years = re.findall(r"[0-9]{4}",date.text)
    if len(years) > 0:
        yearNum.append(int(years[-1]))
        yearStr.append(years[-1])
    else:
        yearNum.append(np.nan)
        yearStr.append('Unknown')

In [14]:
yearNum

[1810, 1830, 1854, 1861, 1870, 1983]

In [123]:
yearNum = [int((yearNum[ii-1]+yearNum[ii+1])/2) if np.isnan(y) else y for ii,y in enumerate(yearNum)]

In [124]:
yearNum

[1810, 1830, 1854, 1861, 1870, 1983]

In [43]:
personStr = []
placesStr = []
for data in soup.find_all("section", {"id":"object-section-provenance"})[0].find_all("div", {"class":"small-12 medium-9 large-9 columns"}):
    cities = []
    countries = []
    print(data.text)
    for city in GeoText(data.text).cities:
        cities.append(city)
    for country in list(GeoText(data.text).country_mentions.keys()):
        countries.append(country)
    if len(countries) == len(cities):
        for t in zip(cities,countries):
            placesStr.append(' '.join(t))
            personStr.append(data.text) 
    elif len(countries) < len(cities):
        for city in cities:
            placesStr.append(city)
            personStr.append(data.text)
    else:
        for country in countries:
            placesStr.append(country)
            personStr.append(data.text)

NameError: name 'soup' is not defined

In [25]:
import inspect
#inspect.getsourcefile(GeoText)
lines = inspect.getsource(GeoText)
print(lines)

class GeoText(object):

    """Extract cities and countries from a text

    Examples
    --------

    >>> places = GeoText("London is a great city")
    >>> places.cities
    "London"

    >>> GeoText('New York, Texas, and also China').country_mentions
    OrderedDict([(u'US', 2), (u'CN', 1)])

    """

    index = build_index()

    def __init__(self, text, country=None):
        city_regex = r"[A-ZÀ-Ú]+[a-zà-ú]+[ \-]?(?:d[a-u].)?(?:[A-ZÀ-Ú]+[a-zà-ú]+)*"
        candidates = re.findall(city_regex, text)
        # Removing white spaces from candidates
        candidates = [candidate.strip() for candidate in candidates]
        self.countries = [each for each in candidates
                          if each.lower() in self.index.countries]
        self.cities = [each for each in candidates
                       if each.lower() in self.index.cities
                       # country names are not considered cities
                       and each.lower() not in self.index.countries]
     

In [2]:
GeoText('Paris').country_mentions

OrderedDict([('FR', 1)])

In [298]:
provhtml = soup.find_all("div", {"class":"detailField provenanceField"})

In [299]:
dischtml = soup.find_all("div", {"class":"detailField webDescriptionField"})

In [300]:
if len(provhtml) >= 2:
    raise ValueError("multiple fields detected")

In [301]:
provStr = provhtml[0].contents[1].text # Contents[0] is the title, [1] should be the data 

In [302]:
provStr = provStr.split('NOTE:')[0].split('\r')[0]

In [303]:
provStr

'By 1990, owned by American Decorative Arts (gallery), Northampton, Massachusetts; January 24, 1990, sold by Chris Kennedy of American Decorative Arts to John Axelrod, Boston, Massachusetts; 2008, promised gift of John Axelrod to the Museum; 2014, gift of Axelrod to the MFA. (Accession date: October 29, 2014)'

In [304]:
provLst = re.split(' to ',provStr)

In [307]:
provLst

['By 1990, owned by American Decorative Arts (gallery), Northampton, Massachusetts; January 24, 1990, sold by Chris Kennedy of American Decorative Arts',
 'John Axelrod, Boston, Massachusetts; 2008, promised gift of John Axelrod',
 'the Museum; 2014, gift of Axelrod',
 'the MFA. (Accession date: October 29, 2014)']

In [306]:
times = []
places = []
addresses = []
coordinates = []
for element in provLst:
    years = re.findall(r"[0-9]{4}", element)
    if not years:
        raise ValueError("year missing")
    times.append(years[-1])
    place = list(dict.fromkeys([re.sub(r'[^\w\s]','',word) for word in element.split() if word[0].isupper() ]))
    place = ' '.join(place)
    places.append(place)
    geocode_result = gmaps.geocode(place)
    addresses.append(re.sub(r',',';',geocode_result[0]['formatted_address']))
    ECo,NCo = geocode_result[0]['geometry']['location'].values()
    coordinates.append((ECo,NCo))

In [308]:
with open('../provenance/'+itemId+'.csv', mode='w') as employee_file:
    prov_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for ii,(time,place,address,coordinate) in enumerate(zip(times,places,addresses,coordinates)):
        prov_writer.writerow([ii,time,place,address,coordinate[0],coordinate[1]])

In [309]:
internalHttp = soup.find_all("img", {"class":"disable-click"})[0]['src']
image_url = 'https://collections.mfa.org/' + internalHttp

In [310]:
try: 
    artist = soup.find_all("div", {"class":"detailField peopleField"})[0].text
except IndexError:
    artist = 'Unknown'

In [311]:
metadata = {'disc': dischtml[0].text, \
            'name' : soup.find_all("h2")[0].text, \
            'by': artist, \
            'image_url': image_url,
            'yearmade' : re.findall(r"[0-9]{4}",soup.find_all("div", {"class":"detailField displayDateField"})[0].text)[-1],
            'tags':tags}

In [312]:
with open('../metadata/'+itemId+'.json', 'w') as outfile:  
    json.dump(metadata, outfile) 

In [292]:
'''
internalHttp = soup.find_all("img", {"class":"disable-click"})[0]['src']
image_url = 'https://collections.mfa.org/' + internalHttp
r = requests.get(image_url, stream = True)
r.raw.decode_content = True
with open('../media/'+itemId+'.jpg', 'wb') as handler:
    handler.write(r.raw.read())
'''

'\ninternalHttp = soup.find_all("img", {"class":"disable-click"})[0][\'src\']\nimage_url = \'https://collections.mfa.org/\' + internalHttp\nr = requests.get(image_url, stream = True)\nr.raw.decode_content = True\nwith open(\'../media/\'+itemId+\'.jpg\', \'wb\') as handler:\n    handler.write(r.raw.read())\n'

In [16]:
from collections import namedtuple, Counter, OrderedDict
import re
import os
import io

In [14]:
def get_data_path(path):
    _ROOT = '/home/sarisadiya/anaconda3/lib/python3.7/site-packages/geotext'
    return os.path.join(_ROOT, 'data', path)

In [10]:
def build_index():
    """Load information from the data directory

    Returns
    -------
    A namedtuple with three fields: nationalities cities countries
    """
    nationalities = read_table(get_data_path('nationalities.txt'), sep=':')
    #print('000000000000')
    
    # parse http://download.geonames.org/export/dump/countryInfo.txt
    countries = read_table(
        get_data_path('countryInfo.txt'), usecols=[4, 0], skip=1)
    #print('1111111111111')

    # parse http://download.geonames.org/export/dump/cities15000.zip
    cities = read_table(get_data_path('cities15000.txt'), usecols=[1, 8])
    #print('2222222222222222')

    # load and apply city patches
    city_patches = read_table(get_data_path('citypatches.txt'))
    cities.update(city_patches)

    Index = namedtuple('Index', 'nationalities cities countries')
    return Index(nationalities, cities, countries)

In [12]:
def read_table(filename, usecols=(0, 1), sep='\t', comment='#', encoding='utf-8', skip=0):
    """Parse data files from the data directory

    Parameters
    ----------
    filename: string
        Full path to file

    usecols: list, default [0, 1]
        A list of two elements representing the columns to be parsed into a dictionary.
        The first element will be used as keys and the second as values. Defaults to
        the first two columns of `filename`.

    sep : string, default '\t'
        Field delimiter.

    comment : str, default '#'
        Indicates remainder of line should not be parsed. If found at the beginning of a line,
        the line will be ignored altogether. This parameter must be a single character.

    encoding : string, default 'utf-8'
        Encoding to use for UTF when reading/writing (ex. `utf-8`)

    skip: int, default 0
        Number of lines to skip at the beginning of the file

    Returns
    -------
    A dictionary with the same length as the number of lines in `filename`
    """

    with io.open(filename, 'r', encoding=encoding) as f:
        # skip initial lines
        for _ in range(skip):
            next(f)

        # filter comment lines
        lines = (line for line in f if not line.startswith(comment))

        d = dict()
        for line in lines:
            columns = line.split(sep)
            key = columns[usecols[0]].lower()
            try:
                value = columns[usecols[1]].rstrip('\n')
            except IndexError:
                print(columns)
            d[key] = value
    return d

In [39]:
index = build_index()

In [16]:
cities = read_table(get_data_path('cities15000.txt'), usecols=[1, 8])

In [45]:
with io.open(get_data_path('citypatches.txt'), 'r', encoding='utf-8') as f:
    sep='\t'
    # skip initial lines
    for _ in range(0):
        next(f)

    # filter comment lines
    lines = (line for line in f if not line.startswith('#'))

    d = dict()
    for line in lines:
        columns = line.split(sep)
        key = columns[0].lower()
        if key == 'paris':
            print(line)
            print('--------------------------')
        value = columns[1].rstrip('\n')
        if key not in d.keys():
            d[key] = value

paris	US

--------------------------


In [43]:
d['paris']

'FR'

In [26]:
city_regex = r"[A-ZÀ-Ú]+[a-zà-ú]+[ \-]?(?:d[a-u].)?(?:[A-ZÀ-Ú]+[a-zà-ú]+)*"
candidates = re.findall(city_regex, 'Paris')

In [27]:
candidates

['Paris']

In [28]:
candidates = [candidate.strip() for candidate in candidates]
countries = [each for each in candidates if each.lower() in index.countries]

In [29]:
countries

[]

In [30]:
candidates

['Paris']

In [31]:
'paris' in index.countries

False

In [32]:
countries = [each for each in candidates if each.lower() in index.countries]
cities = [each for each in candidates if each.lower() in index.cities and each.lower() not in index.countries]

In [33]:
cities

['Paris']

In [34]:
[each for each in candidates if each.lower() in index.nationalities]

[]

In [35]:
[index.countries[country.lower()] for country in countries]

[]

In [36]:
[index.cities[city.lower()] for city in cities]

['US']

In [38]:
index.cities['paris']

'US'

In [ ]:
        self.country_mentions = [self.index.countries[country.lower()]
                                 for country in self.countries]
        self.country_mentions.extend([self.index.cities[city.lower()]
                                      for city in self.cities])
        self.country_mentions.extend([self.index.nationalities[nationality.lower()]
                                      for nationality in self.nationalities])